## A) Read Data Files

### A-1) Pulling in All the Data Across Notebooks

In [128]:
import pandas as pd
import os 
from nltk import tokenize
import json
import re

In [129]:
%store -r HC_reltext
%store -r HC_alltext
%store -r HC_stat

%store -r IND_reltext
%store -r IND_alltext
%store -r IND_stat

%store -r Energy_reltext
%store -r Energy_alltext
%store -r Energy_stat

%store -r CONSTA_reltext
%store -r CONSTA_alltext
%store -r CONSTA_stat

%store -r CONDIS_reltext
%store -r CONDIS_alltext
%store -r CONDIS_stat

%store -r IT_reltext
%store -r IT_alltext
%store -r IT_stat

%store -r Real_Estate_reltext
%store -r Real_Estate_alltext
%store -r Real_Estate_stat

%store -r Materials_reltext
%store -r Materials_alltext
%store -r Materials_stat

%store -r Utilities_reltext
%store -r Utilities_alltext
%store -r Utilities_stat

### Read  <ins>total_relevant</ins>, <ins>total_all</ins>, (both relevant and irrelevant),  <ins>total_stat</ins> (statistics of relevant and all sentences)

In [196]:
total_relevant = pd.concat([HC_reltext, IND_reltext, Energy_reltext,
                 CONSTA_reltext, CONDIS_reltext, IT_reltext,
                 Real_Estate_reltext, Materials_reltext, Utilities_reltext])

In [197]:
total_all = pd.concat([HC_alltext, IND_alltext, Energy_alltext,
                 CONSTA_alltext, CONDIS_alltext, IT_alltext,
                 Real_Estate_alltext, Materials_alltext, Utilities_alltext])

In [198]:
# for i in total_all['all_sentences'].sample(100):
#     print(i)

In [199]:
import numpy as np

unique_comp = []
for i in total_all.company_label:
    if i not in unique_comp:
        unique_comp.append(i)

In [200]:
unique_comp[0:5]

['EliLilly', 'UnitedHealthGroup', 'Merck', 'BristolMyersSquibb', 'Danaher']

In [201]:
comp_dict = dict()

count = 1
for i in unique_comp:
    comp_dict[i] = str("%04d" % count)
    count += 1

In [202]:
comp_dict

{'EliLilly': '0001',
 'UnitedHealthGroup': '0002',
 'Merck': '0003',
 'BristolMyersSquibb': '0004',
 'Danaher': '0005',
 'johnsonandjohnson': '0006',
 'Pfizer': '0007',
 'Abbott': '0008',
 'ThermoFisherScientifiic': '0009',
 'Amgen': '0010',
 'Caterpillar': '0011',
 'Lockheed': '0012',
 'Boeing': '0013',
 'UPS': '0014',
 'Raytheon': '0015',
 'Delta': '0016',
 'Deere': '0017',
 'Honeywell': '0018',
 '3M': '0019',
 'UnionPacific': '0020',
 'Total': '0021',
 'BP': '0022',
 'Shell': '0023',
 'Mondelez_Intl': '0024',
 'Hershey': '0025',
 'Philip_Morris_Intl': '0026',
 'PepsiCo': '0027',
 'Altria_Environmental': '0028',
 'PandG': '0029',
 'Altria_TCFD': '0030',
 'Costco': '0031',
 'CocaCola': '0032',
 'Altria_2021': '0033',
 'Walmart': '0034',
 'EsteeLauder': '0035',
 'McDonalds': '0036',
 'TJX': '0037',
 'HomeDepot': '0038',
 'Lowes': '0039',
 'Target': '0040',
 'BookingHoldings': '0041',
 'Tesla': '0042',
 'Amazon': '0043',
 'Nike': '0044',
 'Starbucks': '0045',
 'Accenture': '0046',
 'Bro

In [203]:
total_all

,all_sentences,company_label
0,"7/7/22, 10:29 AM Environmental | 2021 ESG Repo...",EliLilly
1,Making medicines requires the use of valuable ...,EliLilly
2,We’re committed to reducing our environmental ...,EliLilly
3,"To track our progress, we measure and manage e...",EliLilly
4,"Lilly manages health, safety and the environme...",EliLilly
...,...,...
3567,→ NextEra Energy Resources would invest in el...,NextEraEnergyZeroCarbonBlueprint
3568,→ All non-FPL fossil generation assets would ...,NextEraEnergyZeroCarbonBlueprint
3569,→ Vehicle fleet conversions are based on the ...,NextEraEnergyZeroCarbonBlueprint
3570,Assumptions 0 0,NextEraEnergyZeroCarbonBlueprint


In [204]:
total_all['company_index'] = total_all.apply(lambda x: comp_dict[x.company_label], axis = 1)

In [ ]:
total_all['sent_index'] = total_all.apply(lambda x: comp_dict[x.company_label], axis = 1)

In [205]:
total_all

,all_sentences,company_label,company_index
0,"7/7/22, 10:29 AM Environmental | 2021 ESG Repo...",EliLilly,0001
1,Making medicines requires the use of valuable ...,EliLilly,0001
2,We’re committed to reducing our environmental ...,EliLilly,0001
3,"To track our progress, we measure and manage e...",EliLilly,0001
4,"Lilly manages health, safety and the environme...",EliLilly,0001
...,...,...,...
3567,→ NextEra Energy Resources would invest in el...,NextEraEnergyZeroCarbonBlueprint,0072
3568,→ All non-FPL fossil generation assets would ...,NextEraEnergyZeroCarbonBlueprint,0072
3569,→ Vehicle fleet conversions are based on the ...,NextEraEnergyZeroCarbonBlueprint,0072
3570,Assumptions 0 0,NextEraEnergyZeroCarbonBlueprint,0072


In [206]:
total_all['all_sentences'] = total_all['all_sentences'].apply(lambda x: re.sub(' +', ' ', x))

In [207]:
total_all['all_sentences'] = total_all['all_sentences'].apply(lambda x: re.sub('\xa0', ' ', x))

In [208]:
total_all['all_sentences'] = total_all['all_sentences'].apply(lambda x: re.sub('\t', ' ', x))

In [209]:
total_all['all_sentences'] = total_all['all_sentences'].apply(lambda x: x.strip())

In [210]:
total_all['all_sentences'] = total_all['all_sentences'].apply(lambda x: re.sub('\r', ' ', x))

In [211]:
# for i,j in zip(total_all['all_sentences'], total_all['company_label']) :
#     if i == '(e) High performer is defined as an individual with two consecutive annual performance ratings of Exceeds/Exceeds, Exceeds/Fully Meets, or Fully Meets/Exceeds (note that “exceeds” is one out of four possible dimension ratings).':
#         print(i,j)

In [212]:
total_relevant['company_index'] = total_relevant.apply(lambda x: comp_dict[x.company_label], axis = 1)

In [213]:
total_relevant

,relevant_sentences,company_label,company_index
0,"From 2012 to 2020, we achieved a 26% reduction...",EliLilly,0001
1,"In 2021, we achieved a 9% absolute emissions r...",EliLilly,0001
2,This reduction was partially driven by energy ...,EliLilly,0001
3,"In 2021, 9.6% of our purchased electricity cam...",EliLilly,0001
4,A large portion of this renewable electricity ...,EliLilly,0001
...,...,...,...
67,2018 Retired and demolished 636 MW of coal and...,NextEraEnergyZeroCarbonBlueprint,0072
68,"2019 Aquired Gulf Power, which added 1,750 MW ...",NextEraEnergyZeroCarbonBlueprint,0072
69,2020 Retired 615 MW of nuclear and 330 MW of c...,NextEraEnergyZeroCarbonBlueprint,0072
70,"2021 Added 2,008 MW of wind, 1,547 MW of solar...",NextEraEnergyZeroCarbonBlueprint,0072


In [214]:
total_relevant['relevant_sentences'] = total_relevant['relevant_sentences'].apply(lambda x: re.sub(' +', ' ', x))

In [215]:
total_relevant['relevant_sentences'] = total_relevant['relevant_sentences'].apply(lambda x: re.sub('\xa0', ' ', x))

In [216]:
total_relevant['relevant_sentences'] = total_relevant['relevant_sentences'].apply(lambda x: re.sub('\t', ' ', x))

In [217]:
total_relevant['relevant_sentences'] = total_relevant['relevant_sentences'].apply(lambda x: x.strip())

In [218]:
total_relevant['relevant_sentences'] = total_relevant['relevant_sentences'].apply(lambda x: re.sub('\r', ' ', x))

###  <ins>Regenerate total_stat </ins>after removing duplicate sentences and separating relevant and irreleavant sentences from all sentences

In [220]:
# total_stat  = pd.concat([HC_stat, IND_stat, Energy_stat,
#               CONSTA_stat, CONDIS_stat, IT_stat,
#               Real_Estate_stat, Materials_stat, Utilities_stat])

# total_stat['percentages'] = round(total_stat['percentages'],2)

### A-2) Drop Duplicated Sentences

In [221]:
original_relevant = len(total_relevant)
len(total_relevant.drop_duplicates())
total_relevant = total_relevant.drop_duplicates()
print("relevant_sentences:", original_relevant, "->", len(total_relevant))

relevant_sentences: 1000 -> 998


In [222]:
original_all = len(total_all)
len(total_all.drop_duplicates())
total_all = total_all.drop_duplicates()
print("all_sentences:", original_all, "->", len(total_all))

all_sentences: 81912 -> 79078


## B) Extracting Irrelevant Sentences from All Sentences

### This method below doesn't correctly extract irrelevant sentences. Another method is needed

In [223]:
rel_test = []
for i in total_relevant['relevant_sentences'].to_list():
    if i in total_all['all_sentences'].to_list():
        rel_test.append(i)

In [224]:
print("This method to extract relevant sentences: ", len(rel_test), "vs.", "original_relevant_sentences: ", len(total_relevant['relevant_sentences'].to_list()))

This method to extract relevant sentences:  539 vs. original_relevant_sentences:  998


### Testing fuzz package to compare strings

In [225]:
# pip install fuzzywuzzy
# pip install python-Levenshtein

In [226]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [227]:
fuzz.partial_ratio('In 2021, we reduced our energy consumption by 2.9%,', 
                   'In 2021, we reduced our energy consumption by 2.9%, and we reduced our absolute GHG emissions by 9% compared to 2020.')

100

### B-1) Eliminating short sentences from relevant and all sentences

### Checking the number of letters of sentences to make sure sentences that have less than 5 letters are eliminated

In [228]:
import numpy as np
np.unique([len(i.split(" ")) for i in total_relevant['relevant_sentences']])[0:50]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [229]:
np.unique([len(i.split(" ")) for i in total_all['all_sentences']])[0:50]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [230]:
total_relevant['sent_count'] = total_relevant['relevant_sentences'].str.split().str.len()

/var/folders/c8/4mvtm2vs4_vdgzz_n030prfm0000gn/T/ipykernel_76775/702462658.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_relevant['sent_count'] = total_relevant['relevant_sentences'].str.split().str.len()


In [231]:
total_all['sent_count'] = total_all['all_sentences'].str.split().str.len()

/var/folders/c8/4mvtm2vs4_vdgzz_n030prfm0000gn/T/ipykernel_76775/4064836040.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  total_all['sent_count'] = total_all['all_sentences'].str.split().str.len()


In [232]:
total_relevant = total_relevant[total_relevant['sent_count'] >= 5]

In [233]:
total_all = total_all[total_all['sent_count'] >= 5]

In [234]:
# Check the length of new sentences after dropping duplicates

print("relevant_sentences:", original_relevant, "->", len(total_relevant))
print("all_sentences:", original_all, "->", len(total_all))

relevant_sentences: 1000 -> 957
all_sentences: 81912 -> 77419


### Convert the sentences into lists for future use

In [235]:
rel_var = total_relevant['relevant_sentences'].to_list()
rel_lab = total_relevant['company_label'].to_list()
rel_comp_index = total_relevant['company_index'].to_list()

all_var = total_all['all_sentences'].to_list()
all_lab = total_all['company_label'].to_list()
all_comp_index = total_all['company_index'].to_list()



#### =============================== PAUSE RUNNING HERE ====================================

### B-2) Conduct string matching to separate relevant and irrelevant sentences from all sentences

In [248]:
total_all

,all_sentences,company_label,company_index,sent_count
0,"7/7/22, 10:29 AM Environmental | 2021 ESG Repo...",EliLilly,0001,30
1,Making medicines requires the use of valuable ...,EliLilly,0001,14
2,We’re committed to reducing our environmental ...,EliLilly,0001,18
3,"To track our progress, we measure and manage e...",EliLilly,0001,27
4,"Lilly manages health, safety and the environme...",EliLilly,0001,13
...,...,...,...,...
3565,→ FPL’s four nuclear units continue to operate...,NextEraEnergyZeroCarbonBlueprint,0072,10
3566,Technology We assume that: → FPL’s gas plants ...,NextEraEnergyZeroCarbonBlueprint,0072,28
3567,→ NextEra Energy Resources would invest in ele...,NextEraEnergyZeroCarbonBlueprint,0072,26
3568,→ All non-FPL fossil generation assets would r...,NextEraEnergyZeroCarbonBlueprint,0072,14


In [236]:
# string comparison to separate relevant and irrelevant sentences from all sentences
matrel = []
matlabel = []
orgrel = []
track = 0
for i, j in zip(rel_var, rel_comp_index):
    # extract relevant sentences from all
    val = process.extractOne(i, all_var, scorer = fuzz.partial_ratio, score_cutoff = 85)
    matrel.append(val)
    matlabel.append(j)
    orgrel.append(i)
    print(track, end = " ")
    print(val)
    track +=1

0 ('From 2012 to 2020, we achieved a 26% reduction in absolute emissions.', 100)
1 ('In 2021, we achieved a 9% absolute emissions reduction versus 2020.', 100)
2 ('This reduction was partially driven by energy e�ciency improvements and an increase in the use of renewable electricity including the startup of our solar array in Kinsale.', 100)
3 ('Looking toward the future, we have set climate goals for 2030 as we work toward contributing to a low-carbon economy:  Secure 100% of our purchased electricity from renewable sources In 2021, 9.6% of our purchased electricity came from renewable sources.', 100)
4 ('A large portion of this renewable electricity is delivered through our utility providers to our sites in Alcobendas, Spain; Kinsale, Ireland; and Bracknell, UK.', 100)
5 ('In 2021, we reduced our energy consumption by 2.9%, and we reduced our absolute GHG emissions by 9% compared to 2020.', 100)
6 ('Enhancing the Use of Solar In 2021, 9.6% of our purchased electricity was secured fro

40 ('New this year was the introduction of on-site PPA structure in the form of multiyear “energy as a service” contracts at two sites, enabling GHG-reduction benefits without the need for capital investment.', 100)
41 ('At one of the largest manufacturing sites in South Africa, the Cape Town installation represented the first renewable electricity initiative for Johnson & Johnson in South Africa.', 100)
42 ('Reducing carbon emissions by land, air and sea: As part of our ongoing efforts to leverage data science across our operations, in 2021, we automated a manual freight load planning process to produce a 3D loading plan that optimizes space utilization of shipping containers, resulting in lower GHG emissions from shipping, a faster loading process and cost savings.', 100)
43 ('In Europe, we identified opportunities to switch deliveries from our Belgium distribution center to Denmark, France and Spain from air to road—significantly reducing the carbon emissions with minimal impact to 

81 ('In 2021, we raised the bar with our new 992, which has been demonstrated to produce up to 32% less emissions than its 992K counterpart.', 100)
82 ('CarbonPoint Solutions provides technology to concentrate and capture CO2 for utilization or sequestration, with applications including engines and turbines at oil and gas sites, distributed power and industrial plants, and waste-to-energy sites.', 100)
83 ('CATERPILLAR 2021 SUSTAINABILITY REPORT 57 2018 2019 2020 2021 GOAL 2030 PERSPECTIVE ENERGY — OPERATIONS (continued) Grid electricity % N/A 83.8 87.3 86.7 N/A In 2021, over 35% of our electrical energy was obtained from renewable or alternative sources.', 100)
84 ('Renewable energy 1, 2 % 17.4 21.2 21.0 24.2 N/A Alternative energy 3 % 15.1 14.3 12.0 13.0 N/A Energy intensity /A/ absolute gigajoules energy use/million dollars of sales and revenues 461 445 463 404 N/A In 2021, we reduced our energy intensity by 12% from our 2018 baseline.', 100)
85 ('Absolute greenhouse gas (GHG) emiss

96 ('Net-Zero at Manufacturing and Facilities Boeing achieved net-zero carbon emissions at manufacturing and other facilities and in business travel in 2021 for the second consecutive year, by expanding conservation and renewable energy use while securing verified offsets for the remaining greenhouse gas emissions.', 100)
97 ('Performance Area1 2025 Goals Versus 2017 2021 Progress Toward 2025 Goals3 2030 Goals Greenhouse Gas Emissions Reduce emissions by 25%2 25% reduction Greenhouse gas emissions were 10% under plan primarily due to reduced production activities and procurement of renewables.', 100)
98 ('• Net-zero emissions4 • 55% GHG reduction from 2017 • 100% renewable electricity Energy Reduce energy5 consumption by 10% 12.2% reduction Despite cold northwestern U.S. weather in December, energy continued to be under plan overall for the enterprise, ending the 2021 reporting year at 9.8% under plan.', 100)
99 ('Remote working conditions; reduced production activities; and conservati

146 ('�� Aerodynamic Modifications: Permanent modifications to aircraft to improve their aerodynamic performance.', 100)
147 ('E.g., investments in split-scimitar winglets and blade-tip coating research with the U.S. Air Force.', 100)
148 ('For Delta to reach net zero, out-of-sector solutions will likely be a necessary complement to fleet, SAF, operational initiatives and other in-sector technology improvements, and there will be a need to transition from avoidance (preventing deforestation) and reduction (solar, wind) type projects to removal projects that are absorbing incremental and additional CO2 from the atmosphere.', 100)
149 ('Nature-based solutions such as afforestation and reforestation or technology offsets such as direct air capture and carbon sequestration technologies will be necessary.', 100)
150 ('We allocated a portion of our offset portfolio for 2021 to support climate mitigation efforts that can impact emissions in the near-term, such as through the prevention of def

...

765 ('Also, where possible, we have accelerated grid connections at newly built sites to utilize Kenyan grid power, which is increasingly powered by renewables.', 100)
766 ('As such, we have a goal to expand our renewable energy portfolio to 66 megawatts (MW) of solar capacity installed at over 12,000 sites by 2025.', 100)
767 ('Across our markets, we now have 60 sites that are 100% powered by solar energy, eliminating the need for grid-based electricity or diesel fuel.', 98)
768 ('In 2021, these contracts totaled 213,000 MWh of renewable energy, with approximately 52,000 MWh covering our usage and more than three times that amount, approximately 161,000 MWh, provided to our tenants.', 100)
769 ('In 2021, the South Africa team further expanded our use of renewable energy solutions and installed the Company’s first- ever wind turbine to harness on-site wind power generation.', 100)
770 None
771 ('We more than doubled the number of sites with LIBs compared to 2020, reaching a total of ne

820 ('In 2020, the company sourced 16 million MWh low-carbon energy, or 39 percent of all its purchased electricity.', 100)
821 ('Linde actively sourced 2.4 TWh renewable energy in 2020.', 100)
822 ('Total renewable electricity purchased in 2020 was 2.5 million MWh, or 9.0 million GJ.', 100)
823 ('In 2020, active RE was 2.493 million MWh and passive RE was 9.176 million MWh, for a total of 11.669 million MWh.', 100)
824 ('Nuclear was 4.750 million RE, bringing the total low carbon electricity to 16.419 million MWh of low carbon electricity, which represents 39 percent of Linde’s total electricity consumption in 2020.', 100)
825 None
826 ('305-1 Direct greenhouse gas (GHG) emissions (Scope 1) Linde’s total Scope 1 emissions in 2020 were 16,247,000 MT CO2e, which is a decrease of 1.3 percent from 2019.', 100)
827 ('62 Linde 2020 Sustainable Development Report Scope 1 GHG Emissions 2017 Praxair 2018 Linde Pro Forma 2019 Linde 2020 Linde Scope 1 8,820,000 16,872,000 16,461,000 16,247,000 E

848 ('To date, we have developed two GHG emissions (Scope 1 and 2) reduction targets for our business in order to help manage relevant, climate-related risks and support the decarbonization of our business – the first, established in 2020, seeks to reduce the GHG emissions intensity of Freeport Americas Copper operations by 15% from our 2018 baseline.', 100)
849 ('The second, established this year, seeks to reduce the GHG emissions intensity of our Indonesia operations by 30% by 2030, from our 2018 baseline.', 100)
850 ('ASSET OPTIMIZATION EFFORTS HAVE: Improved the average cycle-time of our haulage fleets at our Americas operations by more than 3 minutes.', 100)
851 ('OR 66,000 METRIC TONS OF CO2 EQUIVALENT, REDUCING OUR ANNUAL AMERICAS SCOPE 1 EMISSIONS BY ~3%.', 100)
852 ('Indonesia As our Grasberg operations transitioned from open-pit to underground mining operations, PT-FI designed and built an autonomous electric train system to move ore through underground tunnels rather than tr

874 ('Newmont Corporation\u20032021 Climate Report 36 Performance, Metrics and Targets\u2002/\u20022050 Carbon-Neutral Goal 2050 Carbon-Neutral Goal Newmont’s goal to be carbon neutral by 2050 is supported by a combination of our long-life portfolio of gold and copper projects with anticipated economic, technological and policy improvements.', 98)
875 ('We will also work concurrently to decarbonize our existing operations, develop and adopt low-carbon technologies, and reconceptualize our greenfield  and brownfield project pipeline as  carbon neutral.', 100)
876 ('Newmont Corporation\u20032021 Climate Report 38 Performance, Metrics and Targets\u2002/\u20022050 Carbon-Neutral Goal CARBON NEUTRALIZATION Achieving our carbon‑neutral goal by 2050 will require carbon removal to neutralize limited “hard to abate”  emissions that cannot yet be eliminated.', 100)
877 ('These carbon removals are commonly referred to as carbon offsets and  such abatement may consider carbon sequestration or capt

910 ('Over 40% of the electricity we generated in 2021 was from carbon-free sources, including nuclear, wind, hydro and solar.', 100)
911 ('Thirty six percent was from lower-carbon natural gas, which emits about half as much carbon dioxide as coal.', 100)
912 ('And about 22% was from higher-carbon coal and oil.', 100)
913 ('Taken together, owned and purchased renewables are equivalent to almost 11% of our electricity generation.', 100)
914 ('- 3 4 35 2.', 88)
915 ('Scope 1 Emissions Emissions From Electric Generation 1 2005 2019 2020 2021 CO2 emissions (thousand metric/short tons) 139,000/ 153,000 84,000/ 93,000 74,000/ 82,000 77,000/ 85,000 CO2 emissions intensity (pounds per net kWh) 1.29 0.86 0.78 0.79 SO2 emissions (metric/short tons) 1,004,000/ 1,107,000 28,000/ 31,000 24,000/ 27,000 23,000/ 25,000 SO2 emissions intensity (pounds per net MWh) 9.3 0.3 0.3 0.2 NOX emissions (metric/short tons) 221,000/ 244,000 45,000/ 50,000 39,000/ 43,000 38,000/ 42,000 NOX emissions intensity (pou

### Check the outcome of string matched sentences and save the files

In [237]:
 string_matched_old = pd.DataFrame([matrel, orgrel, matlabel], index = ['matched_rel', 'original_rel', 'company_label']).transpose()

In [238]:
string_matched = pd.DataFrame([matrel, orgrel, matlabel], index = ['matched_rel', 'original_rel', 'company_label']).transpose()

In [239]:
# extracting sentences portion from the tuple with (sentence, ratio) format
string_matched['matched_rel_only'] = [i[0] if i is not None else '' for i in string_matched.matched_rel]

In [240]:
string_matched.to_csv("string_matched.csv", encoding = 'utf-8-sig')

#### =============================== RESUME RUNNING HERE ====================================

### Keep track of sentences that didn't get matched

In [241]:
string_matched = pd.read_csv('string_matched.csv', index_col = 0)

In [243]:
string_matched

,matched_rel,original_rel,company_label,matched_rel_only
0,"('From 2012 to 2020, we achieved a 26% reducti...","From 2012 to 2020, we achieved a 26% reduction...",1,"From 2012 to 2020, we achieved a 26% reduction..."
1,"('In 2021, we achieved a 9% absolute emissions...","In 2021, we achieved a 9% absolute emissions r...",1,"In 2021, we achieved a 9% absolute emissions r..."
2,('This reduction was partially driven by energ...,This reduction was partially driven by energy ...,1,This reduction was partially driven by energy ...
3,"('Looking toward the future, we have set clima...","In 2021, 9.6% of our purchased electricity cam...",1,"Looking toward the future, we have set climate..."
4,('A large portion of this renewable electricit...,A large portion of this renewable electricity ...,1,A large portion of this renewable electricity ...
...,...,...,...,...
952,('2018 Retired and demolished 636 MW of coal a...,2018 Retired and demolished 636 MW of coal and...,72,2018 Retired and demolished 636 MW of coal and...
953,"('2019 Aquired Gulf Power, which added 1,750 M...","2019 Aquired Gulf Power, which added 1,750 MW ...",72,"2019 Aquired Gulf Power, which added 1,750 MW ..."
954,('2020 Retired 615 MW of nuclear and 330 MW of...,2020 Retired 615 MW of nuclear and 330 MW of c...,72,2020 Retired 615 MW of nuclear and 330 MW of c...
955,"('2021 Added 2,008 MW of wind, 1,547 MW of sol...","2021 Added 2,008 MW of wind, 1,547 MW of solar...",72,"2021 Added 2,008 MW of wind, 1,547 MW of solar..."


In [244]:
still_not_matched = string_matched[string_matched.matched_rel.isnull()]

In [251]:
comp_dict

{'EliLilly': '0001',
 'UnitedHealthGroup': '0002',
 'Merck': '0003',
 'BristolMyersSquibb': '0004',
 'Danaher': '0005',
 'johnsonandjohnson': '0006',
 'Pfizer': '0007',
 'Abbott': '0008',
 'ThermoFisherScientifiic': '0009',
 'Amgen': '0010',
 'Caterpillar': '0011',
 'Lockheed': '0012',
 'Boeing': '0013',
 'UPS': '0014',
 'Raytheon': '0015',
 'Delta': '0016',
 'Deere': '0017',
 'Honeywell': '0018',
 '3M': '0019',
 'UnionPacific': '0020',
 'Total': '0021',
 'BP': '0022',
 'Shell': '0023',
 'Mondelez_Intl': '0024',
 'Hershey': '0025',
 'Philip_Morris_Intl': '0026',
 'PepsiCo': '0027',
 'Altria_Environmental': '0028',
 'PandG': '0029',
 'Altria_TCFD': '0030',
 'Costco': '0031',
 'CocaCola': '0032',
 'Altria_2021': '0033',
 'Walmart': '0034',
 'EsteeLauder': '0035',
 'McDonalds': '0036',
 'TJX': '0037',
 'HomeDepot': '0038',
 'Lowes': '0039',
 'Target': '0040',
 'BookingHoldings': '0041',
 'Tesla': '0042',
 'Amazon': '0043',
 'Nike': '0044',
 'Starbucks': '0045',
 'Accenture': '0046',
 'Bro

In [249]:
still_not_matched

,matched_rel,original_rel,company_label,matched_rel_only
15,NaN,"In 2020, a new solar array was installed at on...",3,NaN
32,NaN,"Total Energy Use (intensity) Reduction 3,696,5...",5,NaN
46,NaN,"From renewable sources 3,355 Percent renewable...",6,NaN
55,NaN,"Energy Consumption by Source(o) 1,000 Gigajoul...",8,NaN
56,NaN,Electricity From Renewable Energy Sources by R...,8,NaN
...,...,...,...,...
860,NaN,"This facility, the largest of its type in the ...",65,NaN
903,NaN,Asheville Plant Gallagher Station Allen Steam ...,70,NaN
906,NaN,Regulated and Commercial Businesses Combined: ...,70,NaN
938,NaN,NextEra has been Energy working,72,NaN


In [259]:
for i in total_all[total_all.company_index == '0052'].all_sentences:
    print(i)
    print()

2021 Environmental Sustainability Report From pledges to progress

Contents Overview Foreword 4 2021 progress 6 How we work 7 Feature story: Employee engagement 9 Commitments and progress 11 About this report 13 Carbon negative Our approach 16 Getting to carbon negative 20 Reducing Scope 1 and 2 emissions 21 Feature story: 100/100/0 carbon-free energy 23 Reducing Scope 3 emissions 25 Removing carbon 30 Driving transformation 31 Enabling systems change 34 Key trends 41 What’s next?

42 Resources 43 Water positive Our approach 45 Getting to water positive 48 Reducing our water footprint 48 Replenishing water 50 Improving access to water 52 Driving transformation 53 Enabling systems change 55 Key trends 58 What’s next?

59 Resources 60 Zero waste Our approach 62 Getting to zero waste 65 Driving transformation 69 Enabling systems change 71 Key trends 74 What’s next?

75 Resources 77 Ecosystems Our approach 79 Taking responsibility for our land footprint 81 Driving transformation 82 Enablin

In [254]:
for i,j in zip(still_not_matched.original_rel, still_not_matched.company_label):
    print(i, "-->", j)
    print()

In 2020, a new solar array was installed at one of our offices in Australia (VPPAs) for utility-scale energy projects based in Texas and Spain. --> 3

Total Energy Use (intensity) Reduction 3,696,540 193.7 -9.0% y/y Total GHG Emissions, Scope 1 and 2 Total GHG Emissions, Scope 1 and 2 (intensity) Reduction 312,860 16.4 -7.5% y/y --> 5

From renewable sources 3,355 Percent renewable electricity by region North America 67%◊ Europe 79%◊ On-site generated energy use by type (TJ)1 Co-generation 456 Wind 111 Solar PV 86 Geothermal 21 Fuel cell 4 Biomass 3 On-site clean/renewable energy capacity by type4 Solar PV Co-generation 44% 29% 23% 2% 2% 1% Fuel cell Biomass Geothermal Wind On-site clean/renewable energy technology capacity, (MW) 67.2 Electricity generated from renewable sources 52%◊ --> 6

Energy Consumption by Source(o) 1,000 Gigajoules Generated Electricity (Cogeneration) Generated Electricity (Renewables) 196 7 Generated On-Site from Renewables 7 Purchased Renewables from Third Par

In [407]:
still_not_matched['sent_count'] = still_not_matched['original_rel'].str.split().str.len()

/var/folders/c8/4mvtm2vs4_vdgzz_n030prfm0000gn/T/ipykernel_36562/842270291.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  still_not_matched['sent_count'] = still_not_matched['original_rel'].str.split().str.len()


In [408]:
still_not_matched.to_csv('sentenences_not_matched.csv', encoding = 'utf-8-sig')

### Keep track of sentences that got matched (only use this version from now on: disregard sentences not matched and short sentences)

In [409]:
sentences_matched = string_matched[string_matched.matched_rel.notnull()]

In [410]:
sentences_matched = sentences_matched.drop_duplicates(subset = ['matched_rel_only', 'company_label'])

In [411]:
sentences_matched.to_csv('sentences_matched.csv', encoding = 'utf-8-sig')

In [412]:
len(sentences_matched)

881

### RUN FROM HERE

### B-4) Separate relevant and irrelevant sentences: aligning matched sentences with total_all sentences

In [413]:
# Reindexing sentences
sentences_matched = pd.read_csv('sentences_matched.csv', index_col = 0)
sentences_matched = sentences_matched.reset_index()
sentences_matched = sentences_matched.drop(columns = ['index'])

In [414]:
all_var_df = pd.DataFrame([all_var, all_lab], index = ['matched_rel_only', 'company_label']).transpose()

In [415]:
sentences_matched = sentences_matched.merge(all_var_df, how = 'right', on = ['matched_rel_only', 'company_label'])

In [416]:
rel_from_all = sentences_matched[sentences_matched.matched_rel.notnull()]

In [417]:
irr_from_all = sentences_matched[sentences_matched.matched_rel.isnull()]

In [418]:
print("Relevant Sentences: ", len(rel_from_all), "| Irrelevant Sentences: ", len(irr_from_all), "| Total: ", len(rel_from_all) + len(irr_from_all))

Relevant Sentences:  871 | Irrelevant Sentences:  77241 | Total:  78112


In [419]:
rel_from_all = rel_from_all.rename(columns = {'original_rel': 'relevant_sentences'})
irr_from_all = irr_from_all.rename(columns = {'matched_rel_only': 'irrelevant_sentences'})

In [420]:
rel_from_all = rel_from_all[['relevant_sentences', 'company_label']]
irr_from_all = irr_from_all[['irrelevant_sentences', 'company_label']]

In [421]:
rel_from_all = rel_from_all.rename(columns = {'relevant_sentences': 'sentences'})

In [422]:
rel_from_all['label'] = 'rel'

In [423]:
irr_from_all = irr_from_all.rename(columns = {'irrelevant_sentences': 'sentences'})

In [424]:
irr_from_all['label'] = 'irr'

In [425]:
all_sent = pd.concat([rel_from_all, irr_from_all])

In [426]:
len(all_sent)

78112

### Clean up extra spaces in sentences to further remove duplicates

In [427]:
all_sent['sentences'] = all_sent['sentences'].apply(lambda x: re.sub(' +', ' ', x))

In [428]:
all_sent['sentences'] = all_sent['sentences'].apply(lambda x: re.sub('\xa0', ' ', x))

In [429]:
all_sent['sentences'] = all_sent['sentences'].apply(lambda x: re.sub('\t', ' ', x))

In [430]:
all_sent['sentences'] = all_sent['sentences'].apply(lambda x: x.strip())

In [431]:
all_sent['sentences'] = all_sent['sentences'].apply(lambda x: re.sub('\r', ' ', x))

In [432]:
all_sent[all_sent.duplicated(subset = 'sentences')]

,sentences,company_label,label
609,• Achieve a 60% reduction in scope 1 and scope...,UnitedHealthGroup,irr
1606,This includes overseeing the company’s program...,Merck,irr
1855,This agreement represents another landmark mil...,Merck,irr
1856,"The global stockpile will offer a critical, ra...",Merck,irr
2093,Prescription Drug Marketing Act and all applic...,Merck,irr
...,...,...,...
75840,Then we will prioritize the categories for whi...,Duke_Energy,irr
75856,These releases are expected to decrease signif...,Duke_Energy,irr
75875,"Building a safe, diverse and engaged workforce.",Duke_Energy,irr
75933,"In December, 71% of enrollments in payment ass...",Duke_Energy,irr


In [433]:
all_sent['sent_no_space'] = all_sent.sentences.str.replace(" ", "")

In [434]:
all_sent = all_sent.drop_duplicates(subset = 'sent_no_space')

In [435]:
all_sent = all_sent.drop_duplicates(subset = 'sentences')

In [436]:
all_sent = all_sent.drop('sent_no_space', axis = 1)

In [437]:
all_sent.index = [x for x in range(1, len(all_sent)+1)]

In [438]:
all_sent['key'] = all_sent.index

In [439]:
all_sent = all_sent.reindex(columns= ['key', 'sentences', 'company_label', 'label'])

In [440]:
all_sent

,key,sentences,company_label,label
1,1,"In 2021, 9.6% of our purchased electricity cam...",EliLilly,rel
2,2,A large portion of this renewable electricity ...,EliLilly,rel
3,3,"From 2012 to 2020, we achieved a 26% reduction...",EliLilly,rel
4,4,"In 2021, we achieved a 9% absolute emissions r...",EliLilly,rel
5,5,This reduction was partially driven by energy ...,EliLilly,rel
...,...,...,...,...
77273,77273,→ FPL’s four nuclear units continue to operate...,NextEraEnergyZeroCarbonBlueprint,irr
77274,77274,Technology We assume that: → FPL’s gas plants ...,NextEraEnergyZeroCarbonBlueprint,irr
77275,77275,→ NextEra Energy Resources would invest in ele...,NextEraEnergyZeroCarbonBlueprint,irr
77276,77276,→ All non-FPL fossil generation assets would r...,NextEraEnergyZeroCarbonBlueprint,irr


In [441]:
all_sent.drop_duplicates('sentences').groupby('label').count()[['key']]

,key
label,
irr,76406
rel,871


### Save extracted_relevant_sentences, extracted_irrelevant_sentences, all_sentences with keys

In [442]:
rel = all_sent[all_sent.label == 'rel']
irr = all_sent[all_sent.label == 'irr']

In [443]:
all_sent.to_csv('all_sentences.csv', encoding = 'utf-8-sig')
rel.to_csv("extracted_relevant_sentences.csv", encoding = 'utf-8-sig')
irr.to_csv("extracted_irrelevant_sentences.csv", encoding = 'utf-8-sig')

In [444]:
all_sent.groupby('label').count()[['key']]

,key
label,
irr,76406
rel,871


## C) Update Sentences Statistics

In [445]:
# rel_from_all = rel_from_all.rename(columns = {'original_rel': 'relevant_sentences'})
# irr_from_all = irr_from_all.rename(columns = {'matched_rel_only': 'irrelevant_sentences'})

In [446]:
# rel_from_all = rel_from_all[['relevant_sentences', 'company_label']]
# irr_from_all = irr_from_all[['irrelevant_sentences', 'company_label']]

In [447]:
rel_from_all = rel.rename(columns = {'sentences': 'relevant_sentences'})

In [448]:
irr_from_all = irr.rename(columns = {'sentences': 'irrelevant_sentences'})

In [449]:
total_relevant_stat = rel_from_all.groupby('company_label', sort = False).count()[['relevant_sentences']]

In [450]:
total_irrelevant_stat = irr_from_all.groupby('company_label', sort = False).count()[['irrelevant_sentences']]

In [451]:
total_stat_final = total_irrelevant_stat.merge(total_relevant_stat, how = 'left', on = 'company_label', sort = False)

In [452]:
total_stat_final['relevant_sentences'] = total_stat_final['relevant_sentences'].fillna(0)

In [453]:
total_stat_final['relevant_sentences'] = total_stat_final['relevant_sentences'].astype(int)

In [454]:
total_stat_final['rel/total percentages'] = round((total_stat_final['relevant_sentences'] / (total_stat_final['relevant_sentences'] + total_stat_final['irrelevant_sentences'])) * 100, 2)

In [455]:
total_stat_final.to_csv('total_stat_final.csv')

In [456]:
total_stat_final.sum()

irrelevant_sentences     76406.00
relevant_sentences         871.00
rel/total percentages      119.52
dtype: float64

### Save the sentence dictionary into json file

In [457]:
sent_dict = {}

for key, sent in zip(all_sent.key, all_sent.sentences):
    sent_dict[key] = sent

In [458]:
import json

with open('sentence_dict.json', 'w') as fp:
    json.dump(sent_dict, fp, sort_keys=True, indent=4)

In [459]:
with open('sentence_dict.json', 'r') as fp:
    data = json.load(fp)

In [460]:
import os
import shutil
import glob

In [461]:
path = '/Users/tylerryoo/t3/extracted_sentences/notebooks/final_extracted_statistics_notebooks'
files = glob.glob(path + "/*.csv")

In [462]:
files.append('/Users/tylerryoo/t3/extracted_sentences/notebooks/final_extracted_statistics_notebooks/sentence_dict.json')

In [463]:
for file in files:
    
    if file == '/Users/tylerryoo/t3/extracted_sentences/notebooks/final_extracted_statistics_notebooks/string_matched.csv':
        continue

    filename = file.split('/')[-1]
    
    target = (r'/Users/tylerryoo/t3/relevant_irrelevant_sentences_labeled_final/' + filename)

    shutil.move(file, target)

### Appendix

In [464]:
#### Previous Method to Separate relevant and irrelevant sentences --> also reordering sentences

In [224]:
# test_extract_relevant = []
# test_extract_relevant_label = []
# test_extract_irrelevant = []
# test_extract_irrelevant_label = []

# for sent, lab in zip(all_var, all_lab):
#     if sent in sentences_matched.matched_rel_only.to_list():
#         test_extract_relevant.append(sent)
#         test_extract_relevant_label.append(lab)
#     elif sent not in sentences_matched.matched_rel_only.to_list():
#         test_extract_irrelevant.append(sent)
#         test_extract_irrelevant_label.append(lab)

In [225]:
# len(test_extract_relevant), len(test_extract_relevant_label)

In [226]:
# len(test_extract_irrelevant), len(test_extract_irrelevant_label)

In [227]:
# ACTION: Make sure to match rel_from_all and extract_rel_var --> to preserve the order of relevant sentences 

In [228]:
# rel_from_all = pd.DataFrame(list(zip(test_extract_relevant, test_extract_relevant_label)), columns = ['relevant_sentences', 'company_label'])

In [229]:
# rel_from_all_reorder = rel_from_all.rename(columns = {'relevant_sentences': 'matched_rel_only'}) 

In [230]:
# rel_from_all_reorder = rel_from_all_reorder.merge(sentences_matched, how = 'left', on = ['matched_rel_only', 'company_label'])

In [231]:
# rel_from_all_reorder[rel_from_all_reorder.company_label == 'Target']

In [232]:
# Observed instances where more null values from original relevant sentences were observed but discard them as they are considered irrelevant
# for i in rel_from_all_reorder[rel_from_all_reorder.original_rel.isnull()]['matched_rel_only']:
#     print(i)

In [233]:
#  #remove null values after merge --> sentences were irrelevant
# rel_from_all_reorder = rel_from_all_reorder[rel_from_all_reorder.original_rel.notnull()]

In [234]:
# rel_from_all_reorder = rel_from_all_reorder[['original_rel', 'matched_rel_only', 'company_label']] 

In [235]:
# irr_from_all = pd.DataFrame(list(zip(test_extract_irrelevant, test_extract_irrelevant_label)),  columns = ['irrelevant_sentences', 'company_label'])

In [236]:
# org_rel = pd.DataFrame(list(zip(sentences_matched.original_rel, sentences_matched.company_label)), columns = ['relevant_sentences', 'company_label'])

In [237]:
# len(rel_from_all_reorder), len(irr_from_all)

In [238]:
# # save the final version of relevant sentences
# pd.DataFrame(rel_from_all).to_csv("extracted_relevant_sentences.csv", encoding = 'utf-8-sig')

In [239]:
# save the final version of irrelevant sentences
# pd.DataFrame(irr_from_all).to_csv("extracted_irrelevant_sentences.csv", encoding = 'utf-8-sig')

In [240]:
# save the final version of relevant sentences
# pd.DataFrame(org_rel).to_csv("extracted_relevant_sentences.csv", encoding = 'utf-8-sig')

In [241]:
# Remove sentences that are shorter than 10 (these are irrelevant sentences)
# still_not_matched = still_not_matched[still_not_matched['sent_count'] >= 10]

In [242]:
# more_match = []
# org_rel = []
# track = 0
# for i, j in zip(still_not_matched.original_rel, still_not_matched.company_label):
#     val = process.extractOne(i.lower(), [i.lower() for i in all_var], scorer = fuzz.partial_ratio)
#     more_match.append(val)
#     org_rel.append(i)
#     print(track, end = " ")
#     track += 1

In [243]:
# more_match_above = []
# for i, j in zip(more_match, org_rel):
#     if i[1] >= 80:
#         print(i)
#         print()
#         print(j)
#         print()
# #         more_match_above.append(i[0])

In [244]:
# extract_rel_var = []
# extract_mat1 = []

# # extracting relevant sentences with ratio greater than 90
# for i, j in zip(mat1, rel_var):
#     if i is None:
#         continue
#     elif i[1] >= 90:
#         extract_rel_var.append(j) # original relevant sentences
#         extract_mat1.append(i) # relevant sentences from all

In [245]:
# extract_mat1 = []
# extract_mat_label = []

# for i, j in zip(mat1, matlabel):
#     if i is None:
#         continue
#     elif i[1] >= 90:
#         extract_mat_label.append(j) # relevant sentences comp labels
#         extract_mat1.append(i) # relevant sentences from all

In [246]:
# len(extract_mat1), len(extract_mat_label), len(extract_rel_var)

In [247]:
# only saving extracted relevant sentences without ratio values  
# rel_from_all = []
# for i in extract_mat1:
#     rel_from_all.append(i[0])